In [33]:
import pandas as pd
import json
import os

In [34]:
def file_read(file):
    print(f'read from {file}.')
    cars_info = []
    camera_id = 0
    with open(file, 'r') as r:
        first = True
        for line in r.readlines():
            camera_result = json.loads(line)
            if first:
                camera_id = camera_result.get('camera_id')
                first = False
            flow_info = camera_result.get('flow_info')
            if flow_info is None:
                continue
            else:
                for car_info in flow_info:
                    cars_info.append(car_info.get('car_info'))
    cars_info = pd.DataFrame(cars_info)
    cars_info['camera_id'] = camera_id
    return cars_info


In [35]:
def get_veh_flow(vehicle_fold, date_sel=None, id_sel=None):
    '''
    date_sel: ['2019-05-13', ...]
    id_sel: ['1', ...]
    '''
    veh_flow = pd.DataFrame()
    all_file = os.listdir(vehicle_fold)
    all_file.sort()
    for file in all_file:
        if file[:7] != 'vehicle':
            continue
        _id, _date = file.split('_')
        _, _id = _id.split('-')
        if date_sel is not None:
            if _date not in date_sel:
                continue
        if id_sel is not None:
            if _id not in id_sel:
                continue

        file = vehicle_fold + '/' + file
        cars_info = file_read(file)

        if len(cars_info) == 0:
            continue

        cars_info.passtime = pd.to_datetime(
            cars_info.passtime, format="%Y-%m-%dT%H:%M:%S")
        veh_flow = veh_flow.append(cars_info)

#     cars_info.rename(columns={'car_id': 'vehicle_id'}, inplace=True)

    return veh_flow


In [36]:
def get_all_veh_flow(vehicle_fold, date_sel=None, id_sel=None):
    '''
    date_sel: ['2019-05-13', ...]
    id_sel: ['1', ...]
    '''
    veh_flow = get_veh_flow(vehicle_fold, date_sel, id_sel)
    print(veh_flow.head())

    print(f'drop_duplicates.')
    veh_flow.drop_duplicates(inplace=True)

    # 匹配camera信息
    print(f'camera_match.')
    lane_info = pd.read_csv('/Users/panbingqing/Documents/工作/城市大脑/长沙调控/data_inline/lane_info.csv')
    camera_match = lane_info[['camera_id','inter_name', 'road_name']].drop_duplicates()
    camera_match.index = camera_match.pop('camera_id')
    camera_match = camera_match.to_dict(orient='index')

    veh_flow['inter_name'] = veh_flow.camera_id.apply(
        lambda x: camera_match[x]['inter_name'])
    veh_flow['road_name'] = veh_flow.camera_id.apply(
        lambda x: camera_match[x]['road_name'])

    # 更新错误车道direction信息
    veh_flow.loc[(veh_flow.camera_id == 9) & (
        veh_flow.lane == 2), 'direction'] = 'SL'

    return veh_flow

In [37]:
if __name__ == '__main__':
    #path =
    veh_flow = get_all_veh_flow( '/Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle')

    print(f'out file.')
    veh_flow.to_csv('/Users/panbingqing/Documents/工作/城市大脑/长沙调控/data/veh_flow_xs.csv', index=False)

read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-11.
read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-12.
read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-13.
read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-14.
read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-15.
read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-16.
read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-17.
read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-18.
read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-19.
read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-20.
read from /Users/panbingqing/Documents/工作/城市大脑/长沙调控/all_vehicle/vehicle-10_2019-05-21.
read from /Users/panbingqing/Documents/工作/城

In [38]:
veh_flow.head()

,car_id,class,confidence,direction,lane,passtime,speed,camera_id,inter_name,road_name
0,401246,unknown,0.935377,S,4,2019-05-11 00:00:26.155800,2.463845,10,星沙-望仙,星沙-望仙南
1,401581,unknown,0.990135,S,4,2019-05-11 00:00:26.305800,11.411998,10,星沙-望仙,星沙-望仙南
2,401255,unknown,0.980352,S,3,2019-05-10 23:59:33.521100,30.241423,10,星沙-望仙,星沙-望仙南
3,401266,unknown,0.953683,S,3,2019-05-11 00:00:24.565800,1.767374,10,星沙-望仙,星沙-望仙南
4,401471,unknown,0.932314,S,3,2019-05-11 00:00:24.687100,3.119720,10,星沙-望仙,星沙-望仙南
